# Aile İlişkileri Ontolojisi

Bu örnek, [Yeni Başlayanlar için Yapay Zeka Müfredatı](http://github.com/microsoft/ai-for-beginners)'nın bir parçasıdır ve [bu blog gönderisinden](https://habr.com/posta/270857/) esinlenilmiştir.

Bir ailedeki insanlar arasındaki farklı ilişkileri hatırlamakta her zaman zorlanırım. Bu örnekte, aile ilişkilerini ve gerçek soy ağacını tanımlayan bir ontoloji alacağız ve daha sonra tüm akrabaları bulmak için otomatik çıkarsamayı nasıl yapabileceğimizi göstereceğiz.

### Soy Ağacını Almak

Örnek olarak, [Romanov Çar Ailesinin](https://en.wikipedia.org/wiki/House_of_Romanov) soy ağacını alacağız. Aile ilişkilerini tanımlamanın en yaygın biçimi [GEDCOM](https://en.wikipedia.org/wiki/GEDCOM)'dur. Romanov soy ağacını GEDCOM formatında alacağız:

In [ ]:
!head -15 ../data/tsars.ged

GEDCOM dosyasını kullanmak için `python-gedcom` kütüphanesini kullanabiliriz:

In [ ]:
import sys
!{sys.executable} -m pip install python-gedcom

Bu kütüphane, dosya ayrıştırmayla ilgili bazı teknik sorunları ortadan kaldırır, ancak yine de bize ağaçtaki tüm bireylere ve ailelere oldukça düşük düzeyde erişim sağlar. Dosyayı şu şekilde ayrıştırabilir ve tüm bireylerin listesini gösterebiliriz:

In [ ]:
from gedcom.parser import Parser
from gedcom.element.individual import IndividualElement
from gedcom.element.family import FamilyElement
g = Parser()
g.parse_file('../data/tsars.ged')

In [ ]:
d = g.get_element_dictionary()
[ (k,v.get_name()) for k,v in d.items() if isinstance(v,IndividualElement)]

İşte aileler hakkında bilgiyi böyle edinebiliriz. Bunun bize bir **tanımlayıcı** listesi verdiğini ve daha fazla netlik istiyorsak bunları adlara dönüştürmemiz gerektiğini unutmayın:

In [ ]:
d = g.get_element_dictionary()
[ (k,[x.get_value() for x in v.get_child_elements()]) for k,v in d.items() if isinstance(v,FamilyElement)]

### Aile Ontolojisini Alma

Şimdi, Semantik Web üçlüleri kümesi olarak tanımlanan [aile ontolojisine](https://raw.githubusercontent.com/blokhin/genealogical-trees/master/data/header.ttl) bir göz atalım. Bu ontoloji, `isUncleOf` (Amcadır), `isCousinOf` (Kuzendir) ve diğerleri gibi ilişkileri tanımlar. Tüm bu ilişkiler `isMotherOf` (Annedir), `isFatherOf` (Babadır), `isBrotherOf` (Erkekkardeştir) ve `isSisterOf` (Kızkardeştir) temel yüklemleri cinsinden tanımlanır. Ontolojiyi kullanarak diğer tüm ilişkileri çıkarmak için otomatik akıl yürütmeyi kullanacağız.

Burada, `isSisterOf` (Kızkardeştir) ve `isParentOf` (Ebeveyndir) (*Teyze/Hala kişinin ebeveyninin kız kardeşidir*) bileşimi olarak tanımlanan `isAuntOf` (Teyzedir) özelliğinin örnek bir tanımı verilmiştir.

```
fhkb:isAuntOf a owl:ObjectProperty ;
    rdfs:domain fhkb:Woman ;
    rdfs:range fhkb:Person ;
    owl:propertyChainAxiom ( fhkb:isSisterOf fhkb:isParentOf ) .
```

In [ ]:
!head -20 ../data/onto.ttl

### Çıkarsama için Ontoloji Oluşturma

Basit olması için, aile ontolojisinden orijinal kuralları ve GEDCOM dosyamızdan bireyler hakkındaki gerçekleri içerecek bir ontoloji dosyası oluşturacağız. GEDCOM dosyasını inceleyeceğiz ve aileler ve bireyler hakkında bilgi çıkaracağız ve bunları üçlülere dönüştüreceğiz.

In [ ]:
!cp ../data/onto.ttl .

gedcom_dict = g.get_element_dictionary()
individuals, marriages = {}, {}

def term2id(el):
    return "i" + el.get_pointer().replace('@', '').lower()

out = open("onto.ttl","a")

for k, v in gedcom_dict.items():
    if isinstance(v,IndividualElement):
        children, siblings = set(), set()
        idx = term2id(v)

        title = v.get_name()[0] + " " + v.get_name()[1]
        title = title.replace('"', '').replace('[', '').replace(']', '').replace('(', '').replace(')', '').strip()

        own_families = g.get_families(v, 'FAMS')
        for fam in own_families:
            children |= set(term2id(i) for i in g.get_family_members(fam, "CHIL"))

        parent_families = g.get_families(v, 'FAMC')
        if len(parent_families):
            for member in g.get_family_members(parent_families[0], "CHIL"): # NB adoptive families i.e len(parent_families)>1 are not considered (TODO?)
                if member.get_pointer() == v.get_pointer():
                    continue
                siblings.add(term2id(member))

        if idx in individuals:
            children |= individuals[idx].get('children', set())
            siblings |= individuals[idx].get('siblings', set())
        individuals[idx] = {'sex': v.get_gender().lower(), 'children': children, 'siblings': siblings, 'title': title}

    elif isinstance(v,FamilyElement):
        wife, husb, children = None, None, set()
        children = set(term2id(i) for i in g.get_family_members(v, "CHIL"))

        try:
            wife = g.get_family_members(v, "WIFE")[0]
            wife = term2id(wife)
            if wife in individuals: individuals[wife]['children'] |= children
            else: individuals[wife] = {'children': children}
        except IndexError: pass
        try:
            husb = g.get_family_members(v, "HUSB")[0]
            husb = term2id(husb)
            if husb in individuals: individuals[husb]['children'] |= children
            else: individuals[husb] = {'children': children}
        except IndexError: pass

        if wife and husb: marriages[wife + husb] = (term2id(v), wife, husb)

for idx, val in individuals.items():
    added_terms = ''
    if val['sex'] == 'f':
        parent_predicate, sibl_predicate = "isMotherOf", "isSisterOf"
    else:
        parent_predicate, sibl_predicate = "isFatherOf", "isBrotherOf"
    if len(val['children']):
        added_terms += " ;\n    fhkb:" + parent_predicate + " " + ", ".join(["fhkb:" + i for i in val['children']])
    if len(val['siblings']):
        added_terms += " ;\n    fhkb:" + sibl_predicate + " " + ", ".join(["fhkb:" + i for i in val['siblings']])
    out.write("fhkb:%s a owl:NamedIndividual, owl:Thing%s ;\n    rdfs:label \"%s\" .\n" % (idx, added_terms, val['title']))

for k, v in marriages.items():
    out.write("fhkb:%s a owl:NamedIndividual, owl:Thing ;\n    fhkb:hasFemalePartner fhkb:%s ;\n    fhkb:hasMalePartner fhkb:%s .\n" % v)

out.write("[] a owl:AllDifferent ;\n    owl:distinctMembers (")
for idx in individuals.keys():
    out.write("    fhkb:" + idx)
for k, v in marriages.items():
    out.write("    fhkb:" + v[0])
out.write("    ) .")
out.close()

In [ ]:
!tail onto.ttl

### Çıkarım Yapmak 

Şimdi bu ontolojiyi çıkarsama ve sorgulama için kullanabilmek istiyoruz. [RDFLib](https://github.com/RDFLib), RDF Çizgesinin farklı formatlarda okunması, sorgulanması vb. için kütüphane kullanacağız.

Mantıksal çıkarım için, RDF Çizgesinin **Kapanışını** oluşturmamıza, yani tüm olası kavramları ve ilişkileri ekleyip çıkarsamamıza, yardımcı olan [OWL-RL](https://github.com/RDFLib/OWL-RL) kütüphanesini kullanacağız.

Ontoloji dosyasını açalım ve kaç tane üçlü içerdiğini görelim:

In [ ]:
import rdflib
from owlrl import DeductiveClosure, OWLRL_Extension

g = rdflib.Graph()
g.parse("onto.ttl", format="turtle")

print("Bununla ucluler:%d" % len(g))

Şimdi kapanışı oluşturalım ve üçlülerin sayısının nasıl arttığını görelim:

In [ ]:
DeductiveClosure(OWLRL_Extension).expand(g)
print("Cikarsama sonrasi ucluler:%d" % len(g))

### Akrabaları Sorgulama

Artık insanlar arasındaki farklı ilişkileri görmek için çizgeyi sorgulayabiliriz. **SPARQL** dilini `query` metodu ile birlikte kullanabiliriz. Bizim durumumuzda, soy ağacımızdaki tüm **amcaları** görelim:

In [ ]:
qres = g.query(
    """SELECT DISTINCT ?aname ?bname
       WHERE {
          ?a fhkb:isUncleOf ?b .
          ?a rdfs:label ?aname .
          ?b rdfs:label ?bname .
       }""")

for row in qres:
    print("%s amcasidir %s" % row)

Farklı diğer aile ilişkilerini denemekten çekinmeyin. Örneğin, belirli bir kişinin tüm atalarını yinelemeli tanımlayan `isAncestorOf` (Atadır) ilişkisine bakabilirsiniz.

Son olarak, etrafı temizleyelim!

In [ ]:
!rm onto.ttl